<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_data_import_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Import Pipeline

This code is still under construction, and is therefore very very bad in places. 

### TODO
- Import CO2 dataset
- Extract unnecessary methods into normal python files and import where necessary. 
- Remove unnecessary variable changes where necessary - this stacks up all the JSON, making everything harder than it needs to be. 
  - Actually, it might not
- Change unnecessary image imports to feature imports 
- Figure out a way of iterating through existing images and displaying the area currently covered by my dataset on a map. 
- Define and import other regions of interest - stick to cities and suburbs for now, since that will have the best health data. Converting this to include rural or rocky areas is an increase in feature set. 
- Figure out how accurate the image exports are
  - Are the points definitely centered on the given coordinates? 
  - is there a way of standardising lighting? 
- add file indexing into one CSV with all our existing latlong exports, so we're not constantly querying the filesystem. 
- list exported files into a CSV 
- Combine GHG exports into one CSV 
- Move data processing methods into their own pipeline 

## Setup
*   Import necessary libraries
*   Import fyputil module
*   Set up Earth Engine authentication and mount google drive  


In [1]:
import ee
import os
import pandas as pd 

from google.colab import drive
# remove this 
from osgeo import gdal
from PIL import Image
from pprint import pprint

In [ ]:
ee.Authenticate()
ee.Initialize()
drive.mount('/content/drive')

In [ ]:
%rm -rf /content/fyp

In [3]:
# Import FYP repo so we can access fyputil common library 
%cd /content
!git clone https://github.com/WRFitch/fyp.git

/content
Cloning into 'fyp'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 1053 (delta 46), reused 20 (delta 11), pack-reused 971
Receiving objects: 100% (1053/1053), 170.13 MiB | 33.66 MiB/s, done.
Resolving deltas: 100% (616/616), done.


In [4]:
# Import fyputil library
%cd fyp/src/fyputil
import constants as c
import ee_constants as eec
import ee_utils as eeutil
import fyp_utils as fyputil
%cd /content

/content/fyp/src/fyputil
/content


# Dataset import

### Import the following datasets into Google Drive

*   [Sentinel-2 Satellite photography](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
*   [Sentinel-5 Precursor Data](https://developers.google.com/earth-engine/datasets/catalog/sentinel)
  *   [Carbon Monoxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CO)
  *   [Formaldehyde](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_HCHO)
  *   [Nitrogen Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2)
  *   [Ozone](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_O3)
  *   [Sulphur Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_SO2)
  *   [Methane](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CH4)
*   [ODIAC Fossil Fuel CO2 Emissions](https://db.cger.nies.go.jp/dataset/ODIAC/DL_odiac2019.html)

##### Import necessary variables

In [5]:
# Define dataset boundaries for britain and london 
great_britain = eec.great_britain
london = eec.london
uxbridge = eec.uxbridge
millennium_dome = eec.millennium_dome
greenwich = eec.greenwich
w_hemisphere = eec.west_hemisphere
e_hemisphere = eec.east_hemisphere

In [6]:
# TODO move this into a CONSTANTS.py file
# Earth engine username, used to import classified image into ee assets folder
USERNAME = 'wrfitch'
OUTPUT_DIR = USERNAME + "/out/"

# Define collections for each dataset to be used 
s2 = eec.s2
s5_CO = eec.s5_CO
s5_HCHO = eec.s5_HCHO
s5_NO2 = eec.s5_NO2
s5_O3 = eec.s5_O3
s5_SO2 = eec.s5_SO2
s5_CH4 = eec.s5_CH4
#TODO import CO2 dataset

CO_band = c.CO_band
HCHO_band = c.HCHO_band
NO2_band = c.NO2_band
O3_band = c.O3_band
SO2_band = c.SO2_band
CH4_band = c.CH4_band

start_date = eec.start_date
end_date = eec.end_date
vis_palette = eec.vis_palette

drive_path = c.drive_path
export_dir = c.export_dir
geotiff_dir = c.geotiff_dir

In [7]:
# Import datasets 

# For easier iteration down the line. I know I'm not supposed to, but google 
# can't tell me what to do, even if it's a good idea!
ghg_imgs = eec.ghg_imgs
ghg_ids = eec.ghg_ids

### Visualise Data

In [ ]:
eec.map 

### Export Data

Exports as .csv tables and GeoTIFF images. 

#### Exporting CSVs

This method of getting the data is very very stupid, but also it does exactly what I need. 

In [ ]:
# Only once this is completed can you move forward and get pictures from these spreadsheets.
for ghg_img in ghg_imgs:
  csv_name = ghg_img.getInfo().get('bands')[0].get('id')
  #exportTableFromImage(ghg_img,london, 1000, export_dir, csv_name)

#### Getting Images From CSV Data

In [ ]:
eeutil.getImgsFromCsv(f"{c.data_dir}/{c.SO2_band}.csv", s2_img)

In [ ]:
#pprint(ee.batch.Task.list())

# Data processing

In [ ]:
fyputil.geotiffToPng("big_geotiff", f"{c.export_dir}/png_224", rm_artifacts=False)
fyputil.moveFilesByExtension("big_geotiff", f"{c.export_dir}/geotiff_224", ".tif")
fyputil.moveFilesByExtension("big_geotiff", f"{c.export_dir}/png_224", ".png")
fyputil.rmConversionArtifacts("big_geotiff", rmTif=False, rmXml=True)

/content/drive/MyDrive/big_geotiff
/content/drive/MyDrive/big_geotiff


In [ ]:
fyputil.geotiffToPng(f"{c.export_dir}/geotiff_224", f"{c.export_dir}/png_224", rm_artifacts=False)

In [ ]:
fyputil.moveFilesByExtension(f"{c.export_dir}/geotiff_224", f"{c.export_dir}/png_224", ".png")
fyputil.rmConversionArtifacts(f"{c.export_dir}/geotiff_224", rmTif=False, rmXml=True)

In [ ]:
# Cleaning up if things go a bit wrong
fyputil.moveFilesByExtension(f"{export_dir}/geotiff", f"{export_dir}/png", ".png")
fyputil.rmConversionArtifacts(f"{export_dir}/geotiff", rmTif=False, rmXml=True)
fyputil.moveFilesByExtension(geotiff_dir, f"{export_dir}/geotiff", ".tif")

In [ ]:
# TODO add these all into one csv file as part of import pipeline. 
fyputil.parseCsvCoords(f"{c.data_dir}/{CO_band}.csv")
fyputil.parseCsvCoords(f"{c.data_dir}/{HCHO_band}.csv")
fyputil.parseCsvCoords(f"{c.data_dir}/{NO2_band}.csv")
fyputil.parseCsvCoords(f"{c.data_dir}/{O3_band}.csv")
fyputil.parseCsvCoords(f"{c.data_dir}/{SO2_band}.csv")
fyputil.parseCsvCoords(f"{c.data_dir}/{CH4_band}.csv")

In [ ]:
# Parse CSVs into pandas dataframes
# TODO rewrite so we aren't deleting columns directly - do it properly! Incorporate these into one csv export in the 
#      output pipeline 
path = c.data_dir
co_df = pd.read_csv(f"{path}/{CO_band}.csv")
del co_df[".geo"]
hcho_df = pd.read_csv(f"{path}/{HCHO_band}.csv")
del hcho_df[".geo"]
no2_df = pd.read_csv(f"{path}/{NO2_band}.csv")
del no2_df[".geo"]
o3_df = pd.read_csv(f"{path}/{O3_band}.csv")
del o3_df[".geo"]
so2_df = pd.read_csv(f"{path}/{SO2_band}.csv")
del so2_df[".geo"]
ch4_df = pd.read_csv(f"{path}/{CH4_band}.csv")
del ch4_df[".geo"]

# Incorporate individual csvs into one ghg dataframe. Badly. 
# TODO fix this so we aren't repeating the same thing over and over
mrg_params = ['longitude', 'latitude', 'system:index']
# somehow this means "intersect". We're taking the intersect so we know we have common values. 
mrg_type = 'inner'

intersect = pd.merge(so2_df, ch4_df, how=mrg_type, on=mrg_params)
intersect = pd.merge(intersect, co_df, how=mrg_type, on=mrg_params)
intersect = pd.merge(intersect, hcho_df, how=mrg_type, on=mrg_params)
intersect = pd.merge(intersect, no2_df, how=mrg_type, on=mrg_params)
intersect = pd.merge(intersect, o3_df, how=mrg_type, on=mrg_params)

print(intersect.shape)
intersect.iloc[0:4] 

In [ ]:
raw_ghg_df = intersect.copy()

for index, row in intersect.iterrows():
  coords = (row.longitude, row.latitude)
  #print(coords)
  filepath = f"{c.data_dir}/png_224/{coords[0]}_{coords[1]}.png"
  if not os.path.isfile(filepath):
    print(f"dropping {filepath} from row {index}")
    # TODO implement this in a way that doesn't suck. 
    # At least unify implementation in a method or management pipeline 
    raw_ghg_df = raw_ghg_df.drop(index=index)

In [ ]:
print(intersect.shape)
print(raw_ghg_df.shape)
raw_ghg_df.iloc[0:10]

In [13]:
raw_ghg_df.to_csv(f"{c.data_dir}/ghgs.csv")